Data cleaning steps:
1. Observe the data set
2. Start cleaning
3. Wired cases to revise the code
4. Reobserver until we satisfy all the cases

Some useful steps or functions:
1. Check Nan values(columns, rows, total), set a threshold to either drop or fillna(N/A and Nan), N/A need to replace first, Nan fill it directly
2. Check Datatype, astype(string to date, string to int or float, cast float …)
3. Check value (values_counts() to see is there a weird case, sometime we need regex expressions to extract valuable data，.replace()
    - https://www.kaggle.com/sophieg/quick-pandas-string-manipulation
    - https://pandas.pydata.org/pandas-docs/stable/user_guide/text.html)
4. Switch the whole data set: using Numpy or Manually, which depends on the situation.
5. Merge or Concat data resources.
6. Groupby 
7. Piviot (a special case of stack)
    - https://nikgrozev.com/2015/07/01/reshaping-in-pandas-pivot-pivot-table-stack-and-unstack-explained-with-pictures/
8. Stack() and Unstack()

In [1]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
import pandas_profiling

In [2]:
# df profiling
def df_profile(df, profile_name):
    profile = pandas_profiling.ProfileReport(df)
    profile.to_file(outputfile=profile_name)

In [3]:
# df dtypes
def print_dtypes(df):
    df_dtypes = df.dtypes.reset_index()
    df_dtypes = df_dtypes.rename(columns={0:'type'})
    df_dtypes = df_dtypes.groupby('type')['index'].apply(list).reset_index()
    df_dtypes['index'] = df_dtypes.apply(lambda x: ",".join(x['index']), axis=1)
    for k,v in df_dtypes.iterrows():
        print(v.values[0])
        print(v.values[1])o
        print('\n')

    #df_num = df.select_dtypes(include = ['float64', 'int64'])
    return df_dtypes

In [ ]:
# check missing rate
def missing_ratio(df, by='column', drop_ratio=0):
    if by=='column':
        ratio = df.isnull().sum()/len(df)*100
    elif by=='row':
        ratio = df.isnull().sum(axis=1)/len(df.columns)*100
    else:
        ratio = round(df.isnull().sum().sum()/df.size,2)
    if drop_ratio==0:
        return ratio
    else:
        return ratio[ratio>drop_ratio].index
# plot missing data

fillna values

1. Nan: fillna(0),

2. Others like N/A: need to replace first then fillna, like replace('N/A', np.nan)

handel missing data:
https://towardsdatascience.com/how-to-handle-missing-data-8646b18db0d4

convert types

> we have object, int64, float64, bool, datetime64, .timedelta[ns], category

1. float64->int64: astype 
2. object -> bool: astype or np.where
3. object(only numbers) -> numbers: astype or pd.to_numeric(or use errors to check object valid or not)
4. string -> datetime: (https://github.com/OrangeC93/Work_Related/blob/master/python/classify_by/pandas-time.ipynb)
    - pd.to_datetime(format), 
    
    - pd.to_timedelta(units='Day'
    
    - pd.PeriodIndex(freq='Q')) -> .to_timestamp(how='start', 'end') which is to cast datetimeIndex to timestamps type, at beginning of period.
    
    - astype('datetime64[M]','datetime64[Y]')
    
    - pd.series.dt.time or dt.date -> object
5. string -> list: import ast
6. categorical(str) -> numerical

In [ ]:
revise string
-  lower, upper, strip, len, split


duplicate
1. df.duplicated()
2. df.drop_duplicates()

In [16]:
# outliers (https://towardsdatascience.com/data-handling-using-pandas-cleaning-and-processing-3aa657dc9418)
def check_outliers(df, col):  
    plt.figure(figsize=(20,5))
    sns.boxplot(x=df[col], color='grey')
    plt.xlabel(col, fontsize=14)
    plt.show()

def cut_outliers_zscore(df, threshold = 3):  
    #z score
    df_num = df.select_dtypes(exclude=['object'])
    print(df_num.shape)
    df_zscore = df_num[(np.abs(stats.zscore(df_num))<threshold).all(axis=1)]
    print("shape after rejecting outliers: %s" %df_zscore.shape)
    
def check_zscore(df, columns):
    from scipy import stats
    for i in columns:
        if True in list(np.abs(stats.zscore(df[i]))>3):
            print(i, ':')
            check_outliers(df, i)

In [17]:
# check value counts
def value_counts_bycol(df,dtype=str):
    for i in df.columns:
        if dtype in [str, int, float]:
            index = df[i].value_counts().index.values
            print(i, ':', [x for x in index if type(x)==dtype])
        else:
            print(i, ':', [df[i].value_counts().index.values])

In [ ]:
# binning Data: pandas.cut( )

# some cases you can use:
https://towardsdatascience.com/a-gentle-introduction-to-exploratory-data-analysis-f11d843b8184